<a href="https://colab.research.google.com/github/ProfessorPatrickSlatraigh/ISQS3344/blob/main/ISQS3344_ch13_SOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sales and Operations Plan Exercises    
**from Heizer's Production and Operations Management, 14th edition**    

<u>**Chapter 13**</u>    


##Housekeeping    

*Imports of libraries and other setup before we start.*    

In [ ]:
# import pandas library for dataframes and other functionality 
import pandas as pd

# import numpy library for arithmetic on arrays    
import numpy as np

# import datetime object from datetime library for date/time logic 
from datetime import datetime  

# import matplotlib library and related objects for plotting    
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()    

import matplotlib.pyplot as pyplot 


##Load the source data file    

*Read a copy of the Example 13-2 Roofing Supplier monthly demand data into the current working directory using a `.CSV` file which has a copy of the original data.*    


In [ ]:
# ask the operating system to copy a file from Github to the local, current working directory 
!curl "https://raw.githubusercontent.com/ProfessorPatrickSlatraigh/data/main/heizer-om_13-2_monthlyroofing.csv" -o heizer-om_13-2_monthlyroofing.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   158  100   158    0     0    957      0 --:--:-- --:--:-- --:--:--   957


##Read Data into a Dataframe    

*Read the `.CSV` file contents into a `demand_df` dataframe.*

In [ ]:
# reading the local .csv file into a dataframe
demand_df = pd.read_csv('heizer-om_13-2_monthlyroofing.csv')

In [ ]:
demand_df.dtypes

month              object
year                int64
expected_demand    object
production_days     int64
dtype: object

In [ ]:
demand_df

##Perform Transformations and Enrichment    


###Transform Expected Demand

*Replace the text in `expected_demand` with the integer equivalent of the text but first remove the ',' commas.*

In [ ]:
# first, replace the commas in the text of `expected_demand` with an empty string
demand_df['expected_demand'] = demand_df['expected_demand'].str.replace(',','')

# then, replace the string values in `expected_demand` with their integer equivalent
demand_df['expected_demand'] = demand_df['expected_demand'].astype(int)

###Calculate Demand per Day    

In [ ]:
# first, calculate the value of `production_days` -- this results in a value of type *float*
demand_df['demand_per_day']  = demand_df['expected_demand'] / demand_df['production_days']

# then, replace the value of `production_days` with the integer equivalent (rounded) 
demand_df['demand_per_day']  = demand_df['demand_per_day'].round().astype(int)

In [ ]:
demand_df

###Create a Datetime Index for Monthly Periods    

In [ ]:
# transform the `year` into its string equivalent
demand_df['year'] = demand_df['year'].astype(str)

In [ ]:
# create a temporary `yearmonth` column by combining the strings from `year` and `month`
demand_df['dash'] = '-'
demand_df['day'] = '-01'
demand_df['yearmonth'] = demand_df.year.str.cat(demand_df.dash) 
demand_df['yearmonth'] = demand_df.yearmonth.str.cat(demand_df.month) 
demand_df['yearmonthday'] = demand_df.yearmonth.str.cat(demand_df.day) 

In [ ]:
# calculate a new column for year_month as datetime    
demand_df['period']=pd.to_datetime(demand_df['yearmonthday'])

In [ ]:
# cleanup the temporary columns 
del demand_df['dash']
del demand_df['day']
del demand_df['yearmonth']
# del demand_df['yearmonthday'] # may want this later to restore a reset index


In [ ]:
# set the `period` datetime as the row index    
demand_df.set_index('period', inplace=True)

#check datatype of index
demand_df.index

###Calculate Level Production     

*Use the average (mean) monthly demand to calculate a steady level of production per period (month).*    


In [ ]:
demand_df['level_production'] = demand_df['demand_per_day'].mean().round()

##Plot the Demand per Production Day by Period (Month)  

*Plot a bar chart of monthly demand per production day along with a line for the mean demand (level_production).*    

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True


# plot the bar chart of demand per production day
pyplot.bar(demand_df.index.month, demand_df["demand_per_day"])

# plot the line of level production (mean demand) per day
pyplot.plot(demand_df.index.month, demand_df["level_production"], marker='*', color='red')

pyplot.legend(['Level Production', 'Monthly Demand'])
pyplot.xlabel('Month')
pyplot.ylabel('Production rate per working day')
pyplot.title('Production Days by Period')

pyplot.show()

##S&OP Approaches    


**Assume the following values...**    


|Description  |  Variable  |  Value ($)  |  Basis  |
|-------------|------------|---------|---------| 
| Inventory carrying cost|  carrying_cost  |  5  | USD/unit per month |
| Subcontracting cost per unit  |  contractor_cost  |  20  |  USD/unit |
| Average pay rate|  avg_pay_rate  |  10   |  USD/hour  |
| Overtime pay rate|  ot_pay_rate  |  17  |  USD/hour >8  |
| Labor-hours to produce a unit|  labor_per_unit  |  1.6  | hours/unit |
| Cost of increasing daily production rate: 
(hiring and training)|  increment_cost  |  300  |  USD/unit |
| Cost of decreasing daily production rate: 
(layoffs)|  reduction_cost  |  600  |  USD/unit| 


In [ ]:
# assign values to variables per the table above
carrying_cost = 5 
contractor_cost = 20
avg_pay_rate = 10
ot_pay_rate = 17
labor_per_unit = 1.6
increment_cost = 300
reduction_cost = 600 

###Plan 1 - Level Monthly Production    

####Calculate labor effort (workers) at level production   

In [ ]:
# calculate daily workforce requirement (workers) using level production and an 8-hour day
plan1_workforce = int((labor_per_unit * 50)/8) 
print(F'The daily labor force required under Plan 1 is {plan1_workforce} workers.')

####Calculate production and inventory values    

In [ ]:
# calculate the plan1_production (monthly) at the level_production
demand_df['plan1_production'] = demand_df['production_days'] * demand_df['level_production']

In [ ]:
# calculate the plan1_inventory_delta as expected_demand - plan1_production
demand_df['plan1_inventory_delta'] = demand_df['plan1_production'] - demand_df['expected_demand']

In [ ]:
demand_df

In [ ]:
print(pd.to_numeric(demand_df.plan1_inventory_delta.shift(1)).fillna(0))

In [ ]:
# demand_df['plan1_inventory_delta'].iloc[0:]
demand_df['plan1_carried'].iloc[1:]

In [ ]:
# initialize plan1_ending_inventory as zero (0)
demand_df['plan1_ending_inventory'] = 0

demand_df['plan1_beginning_inventory'] = pd.to_numeric(demand_df.plan1_inventory_delta.shift(1)).fillna(0)

demand_df['plan1_ending_inventory'] = demand_df.plan1_beginning_inventory + demand_df.plan1_inventory_delta


In [ ]:
### MANY STEPS
# initialize plan1_ending_inventory as zero (0)
demand_df['plan1_ending_inventory'] = 0

# pull last period's inventory delta forward as carried inventory
demand_df['plan1_carried'] = demand_df.plan1_inventory_delta.shift(1)
# replace any carried inventory value which is NaN with a zero (0)
demand_df['plan1_carried'] = pd.to_numeric(demand_df['plan1_carried'], errors='coerce').fillna(0)
# replace any negative carried inventory value with a zero (0)
demand_df['plan1_carried'].mask(demand_df.plan1_carried < 0, 0)

# calculate the ending inventory as the inventory delta plus any carried inventory
demand_df['plan1_ending_inventory'] = demand_df['plan1_inventory_delta'] + demand_df['plan1_carried']

In [ ]:
demand_df.axes

In [ ]:
# calculate the total inventory to carry through the forecast period (total of monthly inventory)
plan1_total_inventory = demand_df.plan1_inventory_delta.sum()
print(F'Total units of inventory carried over from one month to the next is {plan1_total_inventory}.')